<a href="https://colab.research.google.com/github/Crusso22202/Real-Estate-Models/blob/main/WSP_Multifamily_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [28]:
import pandas as pd
import numpy as np
from google.colab import drive
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import yfinance as yf
from fredapi import Fred

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Historical Financial Data**

# Historical Financial Data Import

In [3]:
pd.set_option('display.max_rows', None)
historical_financials_df = pd.read_csv("/content/drive/My Drive/Real Estate Modeling Historical Financials.csv")
historical_financials_df

,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19,Unnamed: 13
0,NaN,Actual,Actual,Actual,Actual,Actual,Actual,Actual,Actual,Actual,Actual,Actual,Actual,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,INCOME:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RENTAL INCOME:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GROSS POTENTIAL,"165,880.72","169,266.04","174,344.02","181,317.78","190,383.67","199,902.85","197,903.82","201,861.90","203,880.52","197,764.10","187,875.90","185,997.14",1.0
5,LOSS TO VACANCY,"-19,070.96","-19,460.16","-20,043.97","-20,845.73","-21,888.01","-22,982.41","-22,752.59","-23,207.64","-23,439.72","-22,736.53","-21,599.70","-21,383.70",2.0
6,LOSS TO LEASE,"-6,490.96","-6,623.43","-6,822.14","-7,095.02","-7,449.77","-7,822.26","-7,744.04","-7,898.92","-7,977.91","-7,738.57","-7,351.64","-7,278.13",1.0
7,UPFRONT CONCESSIONS,-79.65,-81.27,-83.71,-87.06,-91.41,-95.98,-95.02,-96.92,-97.89,-94.96,-90.21,-89.31,1.0
8,BAD DEBT,"-1,149.71","-1,173.17","-1,208.37","-1,256.70","-1,319.54","-1,385.51","-1,371.66","-1,399.09","-1,413.08","-1,370.69","-1,302.15","-1,289.13",4.0
9,MODEL / OFFICE RENT,"-1,128.43","-1,151.46","-1,186.00","-1,233.44","-1,295.12","-1,359.87","-1,346.27","-1,373.20","-1,386.93","-1,345.32","-1,278.06","-1,265.28",3.0


# TOTAL RENTAL INCOME

In [4]:
Total_Rental_Income_df = pd.DataFrame(columns=historical_financials_df.columns)
Total_Rental_Income_df.loc[0, "Unnamed: 0"] = "Net Effective Rent"
Total_Rental_Income_df.loc[1, "Unnamed: 0"] = "Vacancy Loss"
Total_Rental_Income_df.loc[2, "Unnamed: 0"] = "Non-Revenue Units"
Total_Rental_Income_df.loc[3, "Unnamed: 0"] = "Bad Debt"
Total_Rental_Income_df.loc[4, "Unnamed: 0"] = "Total Rental Income"


historical_financials_df.columns = [*historical_financials_df.columns[:-1], 'Category']

# Create a copy of the filtered data to avoid SettingWithCopyWarning
category_1_data = historical_financials_df[historical_financials_df['Category'] == 1].copy()

def calculate_and_store_sums(df, category_number, row_index, target_df):
    # Filter data for the specific category
    category_data = df[df['Category'] == category_number].copy()

    # Clean the data by removing commas and converting to numeric
    category_data.replace(',', '', regex=True, inplace=True)
    for col in category_data.columns[1:-1]:  # Convert all financial columns to numeric
        category_data[col] = pd.to_numeric(category_data[col], errors='coerce')

    # Sum each column and store in the target DataFrame
    for col in category_data.columns[1:-1]:
        total = category_data[col].sum()
        target_df.loc[row_index, col] = total

# Assume historical_financials_df columns are already renamed to include 'Category'

# Calculating sums for each category
calculate_and_store_sums(historical_financials_df, 1, 0, Total_Rental_Income_df)  # Net Effective Rent
calculate_and_store_sums(historical_financials_df, 2, 1, Total_Rental_Income_df)  # Vacancy Loss
calculate_and_store_sums(historical_financials_df, 3, 2, Total_Rental_Income_df)  # Non-Revenue Units
calculate_and_store_sums(historical_financials_df, 4, 3, Total_Rental_Income_df)  # Bad Debt

# Optionally, compute the total rental income as the sum of relevant rows
# This assumes you want to sum the values of Net Effective Rent, Vacancy Loss, etc.
for col in Total_Rental_Income_df.columns[1:-1]:  # Skipping 'Unnamed: 0' and 'Category'
    Total_Rental_Income_df.loc[4, col] = Total_Rental_Income_df.loc[0:3, col].sum()

Total_Rental_Income_df = Total_Rental_Income_df.drop(Total_Rental_Income_df.columns[-1], axis=1)

Total_Rental_Income_df

,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19
0,Net Effective Rent,159310.11,162561.34,167438.17,174135.7,182842.49,191984.61,190064.76,193866.06,195804.72,189930.57,180434.05,178629.7
1,Vacancy Loss,-19070.96,-19460.16,-20043.97,-20845.73,-21888.01,-22982.41,-22752.59,-23207.64,-23439.72,-22736.53,-21599.7,-21383.7
2,Non-Revenue Units,-1128.43,-1151.46,-1186.0,-1233.44,-1295.12,-1359.87,-1346.27,-1373.2,-1386.93,-1345.32,-1278.06,-1265.28
3,Bad Debt,-1149.71,-1173.17,-1208.37,-1256.7,-1319.54,-1385.51,-1371.66,-1399.09,-1413.08,-1370.69,-1302.15,-1289.13
4,Total Rental Income,137961.01,140776.55,144999.83,150799.83,158339.82,166256.82,164594.24,167886.13,169564.99,164478.03,156254.14,154691.59


# TOTAL OTHER REVENUE

In [5]:
Total_Other_Revenue_df = pd.DataFrame(columns=historical_financials_df.columns)
Total_Other_Revenue_df.loc[0, "Unnamed: 0"] = "Utility Reimbursement"
Total_Other_Revenue_df.loc[1, "Unnamed: 0"] = "Other Revenue"
Total_Other_Revenue_df.loc[2, "Unnamed: 0"] = "Total Other Revenue"

# Use the function with Total_Other_Revenue_df
calculate_and_store_sums(historical_financials_df, 5, 0, Total_Other_Revenue_df)  # Utility Reimbursement
calculate_and_store_sums(historical_financials_df, 6, 1, Total_Other_Revenue_df)  # Other Revenue

# Compute the total other revenue by summing the two previous rows
for col in Total_Other_Revenue_df.columns[1:-1]:  # Skipping 'Unnamed: 0' and 'Category'
    Total_Other_Revenue_df.loc[2, col] = Total_Other_Revenue_df.loc[0:1, col].sum()

Total_Other_Revenue_df = Total_Other_Revenue_df.drop(Total_Other_Revenue_df.columns[-1], axis=1)

Total_Other_Revenue_df


,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19
0,Utility Reimbursement,3596.36,3669.75,3779.84,3931.03,4127.59,4333.97,4290.63,4376.43,4420.2,4287.59,4073.22,4032.48
1,Other Revenue,9255.03,9443.93,9727.21,10116.32,10622.11,11153.23,11041.71,11262.54,11375.17,11033.9,10482.2,10377.41
2,Total Other Revenue,12851.39,13113.68,13507.05,14047.35,14749.7,15487.2,15332.34,15638.97,15795.37,15321.49,14555.42,14409.89


# TOTAL INCOME

In [6]:
Total_Income_df = pd.DataFrame(columns=historical_financials_df.columns)
Total_Income_df.loc[0, "Unnamed: 0"] = "TOTAL INCOME"

# Sum the totals from the last rows of both dataframes for each column
for col in Total_Income_df.columns[1:-1]:  # Skip the first descriptive and last category column
    Total_Income_df.loc[0, col] = Total_Rental_Income_df.loc[4, col] + Total_Other_Revenue_df.loc[2, col]

Total_Income_df = Total_Income_df.drop(Total_Income_df.columns[-1], axis=1)

Total_Income_df

,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19
0,TOTAL INCOME,150812.4,153890.23,158506.88,164847.18,173089.52,181744.02,179926.58,183525.1,185360.36,179799.52,170809.56,169101.48


# TOTAL OPERATING EXPENSES

In [7]:
Total_Operating_Expenses_df = pd.DataFrame(columns=historical_financials_df.columns)
Total_Operating_Expenses_df.loc[0, "Unnamed: 0"] = "Repairs & Maintenance"
Total_Operating_Expenses_df.loc[1, "Unnamed: 0"] = "Turnover"
Total_Operating_Expenses_df.loc[2, "Unnamed: 0"] = "Marketing"
Total_Operating_Expenses_df.loc[3, "Unnamed: 0"] = "General & Administrative"
Total_Operating_Expenses_df.loc[4, "Unnamed: 0"] = "Payroll"
Total_Operating_Expenses_df.loc[5, "Unnamed: 0"] = "Management Fees"
Total_Operating_Expenses_df.loc[6, "Unnamed: 0"] = "Utilities"
Total_Operating_Expenses_df.loc[7, "Unnamed: 0"] = "Real Estate Taxes"
Total_Operating_Expenses_df.loc[8, "Unnamed: 0"] = "Property Insurance"
Total_Operating_Expenses_df.loc[9, "Unnamed: 0"] = "Total Operating Expenses"

calculate_and_store_sums(historical_financials_df, 7, 0, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 8, 1, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 9, 2, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 10, 3, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 11, 4, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 12, 5, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 13, 6, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 14, 7, Total_Operating_Expenses_df)
calculate_and_store_sums(historical_financials_df, 15, 8, Total_Operating_Expenses_df)

# Compute the total other revenue by summing the two previous rows
for col in Total_Operating_Expenses_df.columns[1:-1]:  # Skipping 'Unnamed: 0' and 'Category'
    Total_Operating_Expenses_df.loc[9, col] = Total_Operating_Expenses_df.loc[0:9, col].sum()

Total_Operating_Expenses_df = Total_Operating_Expenses_df.drop(Total_Operating_Expenses_df.columns[-1], axis=1)

Total_Operating_Expenses_df

,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19
0,Repairs & Maintenance,4648.23,4743.09,4885.39,5080.81,5334.85,5601.58,5545.57,5656.48,5713.05,5541.65,5264.57,5211.93
1,Turnover,1665.07,1699.05,1750.02,1820.02,1911.02,2006.57,1986.5,2026.23,2046.5,1985.1,1885.85,1866.99
2,Marketing,1328.49,1355.61,1396.27,1452.12,1524.73,1600.97,1584.96,1616.66,1632.82,1583.84,1504.65,1489.6
3,General & Administrative,2798.36,2855.47,2941.13,3058.78,3211.72,3372.31,3338.58,3405.35,3439.4,3336.22,3169.41,3137.72
4,Payroll,6549.0,6682.65,6883.13,7158.45,7516.38,7892.2,7813.27,7969.54,8049.23,7807.76,7417.37,7343.2
5,Management Fees,4524.37,4616.7,4755.2,4945.41,5192.68,5452.32,5397.79,5505.75,5560.81,5393.98,5124.28,5073.04
6,Utilities,3201.88,3267.23,3365.25,3499.86,3674.85,3858.59,3820.0,3896.4,3935.37,3817.31,3626.44,3590.18
7,Real Estate Taxes,25956.46,26486.19,27280.77,28372.0,29790.6,31280.13,30967.33,31586.68,31902.55,30945.47,29398.2,29104.21
8,Property Insurance,2140.52,2184.2,2249.73,2339.72,2456.7,2579.54,2553.74,2604.82,2630.87,2551.94,2424.34,2400.1
9,Total Operating Expenses,52812.38,53890.19,55506.89,57727.17,60613.53,63644.21,63007.74,64267.91,64910.6,62963.27,59815.11,59216.97


# NET OPERATING INCOME

In [8]:
NOI_df = pd.DataFrame(columns=historical_financials_df.columns)
NOI_df.loc[0, "Unnamed: 0"] = "NET OPERATING INCOME"

# Sum the totals from the last rows of both dataframes for each column
for col in NOI_df.columns[1:-1]:  # Skip the first descriptive and last category column
    NOI_df.loc[0, col] = Total_Income_df.loc[Total_Income_df.index[-1], col] - Total_Operating_Expenses_df.loc[Total_Operating_Expenses_df.index[-1], col]

NOI_df = NOI_df.drop(NOI_df.columns[-1], axis=1)

NOI_df

,Unnamed: 0,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19
0,NET OPERATING INCOME,98000.02,100000.04,102999.99,107120.01,112475.99,118099.81,116918.84,119257.19,120449.76,116836.25,110994.45,109884.51


# **INPUTS & DRIVERS**

# Property Overview

In [9]:
Property_Overview_DF = pd.DataFrame(columns=['Property Overview'], index=['Name', 'Type', 'Location', 'Units', 'Year Built'])
units = 150
year_built = 2015
Property_Overview_DF.at['Name', 'Property Overview'] = "Multifamily Property"
Property_Overview_DF.at['Type', 'Property Overview'] = "Multifamily"
Property_Overview_DF.at['Location', 'Property Overview'] = "San Francisco, CA"
Property_Overview_DF.at['Units', 'Property Overview'] = units
Property_Overview_DF.at['Year Built', 'Property Overview'] = year_built

Property_Overview_DF

,Property Overview
Name,Multifamily Property
Type,Multifamily
Location,"San Francisco, CA"
Units,150
Year Built,2015


# Acquisition Assumptions

In [10]:
Acquisition_Assumptions_DF = pd.DataFrame(columns=['Acquisition Assumptions'], index=['Acquisition Date', 'Hold Period (Months)', 'Purchase Price', 'Price Per Unit', 'Acquisition Closing Costs (Percentage)'])
purchase_price = 22500000
hold_period = 60
Acquisition_closing_costs = 0.01
acquisition_date = '12/31/20'
acquisition_date1 = datetime.strptime(acquisition_date, '%m/%d/%y')
Acquisition_Assumptions_DF.at['Acquisition Date', 'Acquisition Assumptions'] = acquisition_date1
Acquisition_Assumptions_DF.at['Hold Period (Months)', 'Acquisition Assumptions'] = hold_period
Acquisition_Assumptions_DF.at['Purchase Price', 'Acquisition Assumptions'] = purchase_price
Acquisition_Assumptions_DF.at['Price Per Unit', 'Acquisition Assumptions'] = Acquisition_Assumptions_DF.at['Purchase Price', 'Acquisition Assumptions'] / units
Acquisition_Assumptions_DF.at['Acquisition Closing Costs (Percentage)', 'Acquisition Assumptions'] = Acquisition_closing_costs
Acquisition_Assumptions_DF

,Acquisition Assumptions
Acquisition Date,2020-12-31 00:00:00
Hold Period (Months),60
Purchase Price,22500000
Price Per Unit,150000.0
Acquisition Closing Costs (Percentage),0.01


# Renovation Budgets

Offensive Renovation Budget

In [11]:
Offensive_Renovation_Budget_DF = pd.DataFrame(columns=['Per Unit', 'Total'], index=['Amount', 'Contingency', 'Subtotal'])
contingency = 0.1
Offensive_Renovation_Budget_DF.at['Amount', 'Total'] = 900000
Offensive_Renovation_Budget_DF.at['Amount', 'Per Unit'] = Offensive_Renovation_Budget_DF.at['Amount', 'Total'] / units
Offensive_Renovation_Budget_DF.at['Contingency', 'Total'] = Offensive_Renovation_Budget_DF.at['Amount', 'Total'] * contingency
Offensive_Renovation_Budget_DF.at['Contingency', 'Per Unit'] = Offensive_Renovation_Budget_DF.at['Contingency', 'Total'] / units
Offensive_Renovation_Budget_DF.at['Subtotal', 'Total'] = Offensive_Renovation_Budget_DF.loc[['Amount', 'Contingency'], 'Total'].sum()
Offensive_Renovation_Budget_DF.at['Subtotal', 'Per Unit'] = Offensive_Renovation_Budget_DF.loc[['Amount', 'Contingency'], 'Per Unit'].sum()


Offensive_Renovation_Budget_DF

,Per Unit,Total
Amount,6000.0,900000
Contingency,600.0,90000.0
Subtotal,6600.0,990000.0


Defensive Renovation Budget

In [12]:
Defensive_Renovation_Budget_DF = pd.DataFrame(columns=['Per Unit', 'Total'], index=['Amount', 'Contingency', 'Subtotal'])
contingency = 0.1
Defensive_Renovation_Budget_DF.at['Amount', 'Total'] = 150000
Defensive_Renovation_Budget_DF.at['Amount', 'Per Unit'] = Defensive_Renovation_Budget_DF.at['Amount', 'Total'] / units
Defensive_Renovation_Budget_DF.at['Contingency', 'Total'] = Defensive_Renovation_Budget_DF.at['Amount', 'Total'] * contingency
Defensive_Renovation_Budget_DF.at['Contingency', 'Per Unit'] = Defensive_Renovation_Budget_DF.at['Contingency', 'Total'] / units
Defensive_Renovation_Budget_DF.at['Subtotal', 'Total'] = Defensive_Renovation_Budget_DF.loc[['Amount', 'Contingency'], 'Total'].sum()
Defensive_Renovation_Budget_DF.at['Subtotal', 'Per Unit'] = Defensive_Renovation_Budget_DF.loc[['Amount', 'Contingency'], 'Per Unit'].sum()

Defensive_Renovation_Budget_DF

,Per Unit,Total
Amount,1000.0,150000
Contingency,100.0,15000.0
Subtotal,1100.0,165000.0


Total Renovation Budget

In [13]:
Total_Renovation_Budget_DF = pd.DataFrame(columns=['Per Unit', 'Total'], index=['Total'])
Total_Renovation_Budget_DF.at['Total', 'Total'] = Defensive_Renovation_Budget_DF.at['Subtotal', 'Total'] + Offensive_Renovation_Budget_DF.at['Subtotal', 'Total']
Total_Renovation_Budget_DF.at['Total', 'Per Unit'] = Total_Renovation_Budget_DF.at['Total', 'Total'] / units

Total_Renovation_Budget_DF

,Per Unit,Total
Total,7700.0,1155000.0


# Unit Mix

In [14]:
Unit_Mix_DF = pd.DataFrame(columns=['Size', 'Count', '% of Total'], index=['Studio', '1BR 1 BA', '2 BR 2BA', '3 BR 2 BA', 'Total'])

Studio_size = 500
onebed_onebath_size = 750
twobed_twobath_size = 1250
threebed_threebath_size = 1500

Studio_count = 15
onebed_onebath_count = 80
twobed_twobath_count = 50
threebed_threebath_count = 5

# Set sizes
Unit_Mix_DF.at['Studio', 'Size'] = Studio_size
Unit_Mix_DF.at['1BR 1 BA', 'Size'] = onebed_onebath_size
Unit_Mix_DF.at['2 BR 2BA', 'Size'] = twobed_twobath_size
Unit_Mix_DF.at['3 BR 2 BA', 'Size'] = threebed_threebath_size

# Set counts
Unit_Mix_DF.at['Studio', 'Count'] = Studio_count
Unit_Mix_DF.at['1BR 1 BA', 'Count'] = onebed_onebath_count
Unit_Mix_DF.at['2 BR 2BA', 'Count'] = twobed_twobath_count
Unit_Mix_DF.at['3 BR 2 BA', 'Count'] = threebed_threebath_count

# Calculate total size and count
total_size = (Studio_size * Studio_count) + (onebed_onebath_size * onebed_onebath_count) + (twobed_twobath_size * twobed_twobath_count) + (threebed_threebath_size * threebed_threebath_count)
total_count = Studio_count + onebed_onebath_count + twobed_twobath_count + threebed_threebath_count

# Calculate percentage of total
Unit_Mix_DF['% of Total'] = (Unit_Mix_DF['Count']/ total_count) * 100

# Set total row
Unit_Mix_DF.at['Total', 'Size'] = total_size
Unit_Mix_DF.at['Total', 'Count'] = total_count
Unit_Mix_DF.at['Total', '% of Total'] = 100.0


Unit_Mix_DF

,Size,Count,% of Total
Studio,500,15,10.0
1BR 1 BA,750,80,53.333333
2 BR 2BA,1250,50,33.333333
3 BR 2 BA,1500,5,3.333333
Total,137500,150,100.0


# Financing Assumptions

In [15]:
Financing_Assumptions_DF = pd.DataFrame(columns=[''], index=['Start Date', 'Term (Months)', 'Maturity Date', 'Loan-To-Value', 'Loan Amount', 'Interest Rate Spread (Basis Points)', 'All-in Rate', 'Fixed Rate', 'Interest Only Period (Months)', 'Amortization Period (Months)', 'Amortization Rate', 'Origination Fee', 'Loan Type'])

loan_term = 60
maturity_date = acquisition_date1 + relativedelta(months=loan_term)
LTV = 0.65
Loan_Amount = purchase_price * LTV
IR_Spread = 300
Fixed_Rate = 0.05
Interest_Only_Period = 24
Amortization_Period = 360
Amortization_Rate = 0.05
Origination_Fee = 0.01
loan_type = "Fixed"


Financing_Assumptions_DF.at['Start Date', ''] = acquisition_date1
Financing_Assumptions_DF.at['Term (Months)', ''] = loan_term
Financing_Assumptions_DF.at['Maturity Date', ''] = maturity_date
Financing_Assumptions_DF.at['Loan-To-Value', ''] = LTV
Financing_Assumptions_DF.at['Loan Amount', ''] = Loan_Amount
Financing_Assumptions_DF.at['Interest Rate Spread (Basis Points)', ''] = IR_Spread
Financing_Assumptions_DF.at['Fixed Rate', ''] = Fixed_Rate
Financing_Assumptions_DF.at['Interest Only Period (Months)', ''] = Interest_Only_Period
Financing_Assumptions_DF.at['Amortization Period (Months)', ''] = Amortization_Period
Financing_Assumptions_DF.at['Amortization Rate', ''] = Amortization_Rate
Financing_Assumptions_DF.at['Origination Fee', ''] = Origination_Fee
Financing_Assumptions_DF.at['Loan Type', ''] = loan_type


Financing_Assumptions_DF

,
Start Date,2020-12-31 00:00:00
Term (Months),60
Maturity Date,2025-12-31 00:00:00
Loan-To-Value,0.65
Loan Amount,14625000.0
Interest Rate Spread (Basis Points),300
All-in Rate,NaN
Fixed Rate,0.05
Interest Only Period (Months),24
Amortization Period (Months),360


# SOFR Data Import

In [16]:
# Initialize the Fred API
fred = Fred(api_key='2e7abe33c05598d3d63fe6ac38b35298')  # Replace 'your_api_key_here' with your FRED API key

# Retrieve historical data for SOFR
sofr_data = fred.get_series('SOFR')  # Secured Overnight Financing Rate (SOFR)
sofr_data = sofr_data.resample('M').last()


# Calculate the next sixty months with values incremented by 0.01
last_date = sofr_data.index[-1]
next_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=Amortization_Period, freq='M')
next_values = [sofr_data.iloc[-1] + i * 0.01 for i in range(Amortization_Period)]

# Create a Series for the next sixty months
next_sofr_data = pd.Series(next_values, index=next_dates)

# Concatenate the existing SOFR data with the next sixty months
sofr_data = pd.concat([sofr_data, next_sofr_data])
sofr_data = pd.DataFrame(sofr_data, columns = ['SOFR'])
sofr_data['All-in Rate'] = sofr_data['SOFR'] + (IR_Spread / 100)

sofr_data

,SOFR,All-in Rate
2018-04-30,1.77,4.77
2018-05-31,1.81,4.81
2018-06-30,2.12,5.12
2018-07-31,1.93,4.93
2018-08-31,1.97,4.97
2018-09-30,2.25,5.25
2018-10-31,2.22,5.22
2018-11-30,2.28,5.28
2018-12-31,3.00,6.00
2019-01-31,2.58,5.58


# Amortization Table

In [27]:
start_date = pd.to_datetime(acquisition_date, format='%m/%d/%y')

def build_amortization_table(Loan_Amount, Amortization_Rate, Amortization_Period, Interest_Only_Period, start_date):
    # Convert annual interest rate to monthly rate
    monthly_interest_rate = Amortization_Rate / 12

    # Calculate total number of payments (amortization period in months)
    total_payments = Amortization_Period

    # Calculate the loan payment using the formula
    loan_payment = Loan_Amount * (monthly_interest_rate * (1 + monthly_interest_rate) ** total_payments) / ((1 + monthly_interest_rate) ** total_payments - 1)

    # Initialize variables for the amortization table
    remaining_balance = Loan_Amount
    amortization_table = []

    # Add period 0 with the initial loan amount as the beginning balance
    amortization_table.append({
        'Date': start_date,
        'Period': 0,
        'Beginning Balance': Loan_Amount,
        'Principal Payment': 0,
        'Interest Payment': 0,
        'Total Payment': 0,  # No payments in period 0
        'Ending Balance': Loan_Amount
    })

    # Loop through each period (month) of the loan term
    for period in range(1, total_payments + 1):  # Start from period 1
        # Check if the current period is within the interest-only period
        if period <= Interest_Only_Period:
            # Only interest payment, no principal payment
            days_in_month = pd.Period(start_date + pd.DateOffset(months=period), 'M').days_in_month
            interest_payment = (Amortization_Rate / Amortization_Period) * days_in_month * remaining_balance
            principal_payment = 0
        else:
            # Regular payment with both principal and interest
            days_in_month = pd.Period(start_date + pd.DateOffset(months=period), 'M').days_in_month
            principal_payment = loan_payment - interest_payment

        # Update the remaining balance
        remaining_balance -= principal_payment

        # Record the values in the amortization table
        amortization_table.append({
            'Date': start_date + pd.DateOffset(months=period),
            'Period': period,
            'Beginning Balance': remaining_balance + principal_payment,
            'Principal Payment': principal_payment,
            'Interest Payment': interest_payment,
            'Total Payment': principal_payment + interest_payment,
            'Ending Balance': remaining_balance
        })

    return amortization_table

# Using start_date from your provided code
start_date = pd.to_datetime(acquisition_date, format='%m/%d/%y')

# Build the amortization table
amortization_table = build_amortization_table(Loan_Amount, Amortization_Rate, Amortization_Period, Interest_Only_Period, start_date)

# Convert the amortization table to a DataFrame
amortization_df = pd.DataFrame(amortization_table)

# Set the 'Date' column as the index
amortization_df.set_index('Date', inplace=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

amortization_df



,Period,Beginning Balance,Principal Payment,Interest Payment,Total Payment,Ending Balance
Date,,,,,,
2020-12-31,0,14625000.00,0.00,0.00,0.00,14625000.00
2021-01-31,1,14625000.00,0.00,62968.75,62968.75,14625000.00
2021-02-28,2,14625000.00,0.00,56875.00,56875.00,14625000.00
2021-03-31,3,14625000.00,0.00,62968.75,62968.75,14625000.00
2021-04-30,4,14625000.00,0.00,60937.50,60937.50,14625000.00
2021-05-31,5,14625000.00,0.00,62968.75,62968.75,14625000.00
2021-06-30,6,14625000.00,0.00,60937.50,60937.50,14625000.00
2021-07-31,7,14625000.00,0.00,62968.75,62968.75,14625000.00
2021-08-31,8,14625000.00,0.00,62968.75,62968.75,14625000.00


# CAP EX SCHEDULE

In [18]:
Cap_Ex_Schedule_DF = pd.DataFrame(columns=['Start Date', 'Duration (Months)', 'End Date'], index=['Defensive', 'Offensive'])

Defensive_Start_Date = datetime.strptime('1/31/21', '%m/%d/%y')
Defensive_Duration = 6
Defensive_End_Date = Defensive_Start_Date + relativedelta(months=Defensive_Duration)
Offensive_Start_Date = datetime.strptime('3/31/21', '%m/%d/%y')
Offensive_Duration = 12
Offensive_End_Date = Offensive_Start_Date + relativedelta(months=Offensive_Duration)

# Set values in DataFrame
Cap_Ex_Schedule_DF.at['Defensive', 'Start Date'] = Defensive_Start_Date
Cap_Ex_Schedule_DF.at['Defensive', 'Duration (Months)'] = Defensive_Duration
Cap_Ex_Schedule_DF.at['Defensive', 'End Date'] = Defensive_End_Date
Cap_Ex_Schedule_DF.at['Offensive', 'Start Date'] = Offensive_Start_Date
Cap_Ex_Schedule_DF.at['Offensive', 'Duration (Months)'] = Offensive_Duration
Cap_Ex_Schedule_DF.at['Offensive', 'End Date'] = Offensive_End_Date

Cap_Ex_Schedule_DF

,Start Date,Duration (Months),End Date
Defensive,2021-01-31 00:00:00,6,2021-07-31 00:00:00
Offensive,2021-03-31 00:00:00,12,2022-03-31 00:00:00


# UNIT RENOVATIONS

In [19]:
Unit_Renovations_DF = pd.DataFrame(columns=[''], index=['Units Renovated / Month', 'Cost / Unit', 'Rent Premium'])

Units_Renovated_Month = units / Offensive_Duration
Cost_per_unit = 6000
Rent_Premium = 125

Unit_Renovations_DF.at['Units Renovated / Month', ''] = Units_Renovated_Month
Unit_Renovations_DF.at['Cost / Unit', ''] = Cost_per_unit
Unit_Renovations_DF.at['Rent Premium', ''] = Rent_Premium

Unit_Renovations_DF



,
Units Renovated / Month,12.50
Cost / Unit,6000
Rent Premium,125


# JOINT VENTURE ASSUMPTIONS

In [20]:
JV_Assumptions_DF = pd.DataFrame(columns=['Promote', 'Pref', 'Splits'], index=['Hurdle #1', 'Hurdle #2', 'Hurdle #3', 'Hurdle #4'])

GP_Equity = 0.1

Promote_Hurdle2 = 0.15
Promote_Hurdle3 = 0.25
Promote_Hurdle4 = 0.4

Pref_Hurdle1 = 0.12
Pref_Hurdle2 = 0.16
Pref_Hurdle3 = 0.2
Pref_Hurdle4 = 5.0

Splits_Hurdle1 = GP_Equity
Splits_Hurdle2 = GP_Equity + ((1-GP_Equity)*Promote_Hurdle2)
Splits_Hurdle3 = GP_Equity + ((1-GP_Equity)*Promote_Hurdle3)
Splits_Hurdle4 = GP_Equity + ((1-GP_Equity)*Promote_Hurdle4)

JV_Assumptions_DF.at['Hurdle #2','Promote'] = Promote_Hurdle2
JV_Assumptions_DF.at['Hurdle #3','Promote'] = Promote_Hurdle3
JV_Assumptions_DF.at['Hurdle #4','Promote'] = Promote_Hurdle4
JV_Assumptions_DF.at['Hurdle #1','Pref'] = Pref_Hurdle1
JV_Assumptions_DF.at['Hurdle #2','Pref'] = Pref_Hurdle2
JV_Assumptions_DF.at['Hurdle #3','Pref'] = Pref_Hurdle3
JV_Assumptions_DF.at['Hurdle #4','Pref'] = Pref_Hurdle4
JV_Assumptions_DF.at['Hurdle #1','Splits'] = Splits_Hurdle1
JV_Assumptions_DF.at['Hurdle #2','Splits'] = Splits_Hurdle2
JV_Assumptions_DF.at['Hurdle #3','Splits'] = Splits_Hurdle3
JV_Assumptions_DF.at['Hurdle #4','Splits'] = Splits_Hurdle4
JV_Assumptions_DF

,Promote,Pref,Splits
Hurdle #1,NaN,0.12,0.10
Hurdle #2,0.15,0.16,0.24
Hurdle #3,0.25,0.20,0.33
Hurdle #4,0.40,5.00,0.46


# EXIT ASSUMPTIONS

In [21]:
Exit_Assumptions_DF = pd.DataFrame(columns=[''], index=['Exit Date', 'Exit Cap Rate', 'Sale Price', 'Exit Closing Costs'])

Exit_Date = acquisition_date1 + relativedelta(months=hold_period)
Exit_Cap_Rate = 0.06
Exit_Closing_Costs = 0.01

Exit_Assumptions_DF.at['Exit Date', ''] = Exit_Date
Exit_Assumptions_DF.at['Exit Cap Rate', ''] = Exit_Cap_Rate
Exit_Assumptions_DF.at['Exit Closing Costs', ''] = Exit_Closing_Costs

Exit_Assumptions_DF

,
Exit Date,2025-12-31 00:00:00
Exit Cap Rate,0.06
Sale Price,NaN
Exit Closing Costs,0.01


# REVENUE & EXPENSE ASSUMPTIONS

In [22]:
Rev_Exp_Assumptions_DF = pd.DataFrame(columns=['Year', 'Occupancy', 'Rent Growth', 'Other Revenue Growth', 'Expense Growth', 'RET Growth' ], index=[])

years = range(1, 12)
Rev_Exp_Assumptions_DF['Year'] = years

initial_occupancy = 0.90
target_occupancy = 0.93
growth_rate = 0.01

def calculate_occupancy(year):
    return min(initial_occupancy + growth_rate * (year - 1), target_occupancy)

Rev_Exp_Assumptions_DF['Occupancy'] = [calculate_occupancy(year) for year in years]

rent_growth_year1 = 0.0
rent_growth_year2 = 0.02
rent_growth_year3 = 0.03

Rev_Exp_Assumptions_DF['Rent Growth'] = [
    rent_growth_year1 if year == 1 else
    rent_growth_year2 if year == 2 else
    rent_growth_year3 if year == 3 else
    rent_growth_year3
    for year in years
]

Other_revenue_growth_year1 = 0.0
Other_revenue_growth_year2 = 0.03
Other_revenue_growth_year3 = 0.03

Rev_Exp_Assumptions_DF['Other Revenue Growth'] = [
    Other_revenue_growth_year1 if year == 1 else
    Other_revenue_growth_year2 if year == 2 else
    Other_revenue_growth_year3 if year == 3 else
    Other_revenue_growth_year3
    for year in years
]

Expense_growth_year1 = 0.0
Expense_growth_year2 = 0.02
Expense_growth_year3 = 0.03

Rev_Exp_Assumptions_DF['Expense Growth'] = [
    Expense_growth_year1 if year == 1 else
    Expense_growth_year2 if year == 2 else
    Expense_growth_year3 if year == 3 else
    Expense_growth_year3
    for year in years
]

RET_growth_year1 = 0.0
RET_growth_year2 = 0.03
RET_growth_year3 = 0.03

# Set the "Rent Growth" column values
Rev_Exp_Assumptions_DF['RET Growth'] = [
    RET_growth_year1 if year == 1 else
    RET_growth_year2 if year == 2 else
    RET_growth_year3 if year == 3 else
    RET_growth_year3
    for year in years
]

Rev_Exp_Assumptions_DF

,Year,Occupancy,Rent Growth,Other Revenue Growth,Expense Growth,RET Growth
0,1,0.90,0.00,0.00,0.00,0.00
1,2,0.91,0.02,0.03,0.02,0.03
2,3,0.92,0.03,0.03,0.03,0.03
3,4,0.93,0.03,0.03,0.03,0.03
4,5,0.93,0.03,0.03,0.03,0.03
5,6,0.93,0.03,0.03,0.03,0.03
6,7,0.93,0.03,0.03,0.03,0.03
7,8,0.93,0.03,0.03,0.03,0.03
8,9,0.93,0.03,0.03,0.03,0.03
9,10,0.93,0.03,0.03,0.03,0.03


# IN PLACE INCOME T-3



In [23]:
Annual_total_rental_income = Total_Rental_Income_df.iloc[:, -3:].sum(axis=1) * 4
Annual_other_revenue_income = Total_Other_Revenue_df.iloc[:2, -3:].sum(axis=1) * 4
Annual_total_income = Total_Income_df.iloc[:, -3:].sum(axis=1) * 4


Annual_total_rental_income_DF = pd.DataFrame(Annual_total_rental_income)
Annual_total_rental_income_DF.columns = ['Total']
Annual_total_rental_income_DF.index = ['Net Effective Rent', 'Vacancy Loss', 'Non-Revenue Units', 'Bad Debt', 'Total Rental Income']

annual_other_revenue_income_df = pd.DataFrame(Annual_other_revenue_income)
annual_other_revenue_income_df.columns = ['Total']
annual_other_revenue_income_df.index = ['Utility Reimbursement', 'Other Income']

Annual_total_income_df = pd.DataFrame(Annual_total_income)
Annual_total_income_df.columns = ['Total']
Annual_total_income_df.index = ['Total Income']

Annual_total_income_DF = pd.concat([Annual_total_rental_income_DF, annual_other_revenue_income_df, Annual_total_income_df], axis=0)

Annual_total_income_DF

,Total
Net Effective Rent,2195977.28
Vacancy Loss,-262879.72
Non-Revenue Units,-15554.64
Bad Debt,-15847.88
Total Rental Income,1901695.04
Utility Reimbursement,49573.16
Other Income,127574.04
Total Income,2078842.24


# IN-PLACE EXPENSES

In [26]:
In_Place_Expenses_DF = pd.DataFrame(columns=['Per Unit', 'Total' ], index=['Repairs & Maintenance', 'Turnover', 'Marketing', 'General & Administrative', 'Payroll', 'Management Fees', 'Utilities', 'Real Estate Taxes', 'Property Insurance', 'Capital Reserves', ' Total'])

Repairs_Maintenance_PU = 400
Turnover_PU = 150
Marketing_PU = 150
General_Admin_PU = 250
Payroll_PU = 600
Management_Fees = 0.03
Utilities_PU = 300
Property_Insurance_PU = 200
Capital_Reserves_PU = 200

Repairs_Maintenance_Total = Repairs_Maintenance_PU * units
Turnover_Total = Turnover_PU * units
Marketing_Total = Marketing_PU * units
General_Admin_Total = General_Admin_PU * units
Payroll_Total = Payroll_PU * units
Management_Fees_Total = Annual_total_income_DF.loc["Total Rental Income"].values[0] * Management_Fees
Management_Fees_PU = Management_Fees_Total / units
Utilities_Total = Utilities_PU * units
Real_Estate_Taxes_Total = 400000
Real_Estate_Taxes_PU = Real_Estate_Taxes_Total / units
Property_Insurance_Total = Property_Insurance_PU * units
Capital_Reserves_Total = Capital_Reserves_PU * units

# Fill in the DataFrame
In_Place_Expenses_DF.at['Repairs & Maintenance', 'Per Unit'] = Repairs_Maintenance_PU
In_Place_Expenses_DF.at['Turnover', 'Per Unit'] = Turnover_PU
In_Place_Expenses_DF.at['Marketing', 'Per Unit'] = Marketing_PU
In_Place_Expenses_DF.at['General & Administrative', 'Per Unit'] = General_Admin_PU
In_Place_Expenses_DF.at['Payroll', 'Per Unit'] = Payroll_PU
In_Place_Expenses_DF.at['Management Fees', 'Per Unit'] = Management_Fees_PU
In_Place_Expenses_DF.at['Utilities', 'Per Unit'] = Utilities_PU
In_Place_Expenses_DF.at['Real Estate Taxes', 'Per Unit'] = Real_Estate_Taxes_PU
In_Place_Expenses_DF.at['Property Insurance', 'Per Unit'] = Property_Insurance_PU
In_Place_Expenses_DF.at['Capital Reserves', 'Per Unit'] = Capital_Reserves_PU
In_Place_Expenses_DF.at[' Total', 'Per Unit'] = In_Place_Expenses_DF['Total'].sum()



# Fill in the Total row
In_Place_Expenses_DF.at['Repairs & Maintenance', 'Total'] = Repairs_Maintenance_Total
In_Place_Expenses_DF.at['Turnover', 'Total'] = Turnover_Total
In_Place_Expenses_DF.at['Marketing', 'Total'] = Marketing_Total
In_Place_Expenses_DF.at['General & Administrative', 'Total'] = General_Admin_Total
In_Place_Expenses_DF.at['Payroll', 'Total'] = Payroll_Total
In_Place_Expenses_DF.at['Management Fees', 'Total'] = Management_Fees_Total
In_Place_Expenses_DF.at['Utilities', 'Total'] = Utilities_Total
In_Place_Expenses_DF.at['Real Estate Taxes', 'Total'] = Real_Estate_Taxes_Total
In_Place_Expenses_DF.at['Property Insurance', 'Total'] = Property_Insurance_Total
In_Place_Expenses_DF.at['Capital Reserves', 'Total'] = Capital_Reserves_Total
In_Place_Expenses_DF.at[' Total', 'Total'] = In_Place_Expenses_DF['Total'].sum()
In_Place_Expenses_DF.at[' Total', 'Per Unit'] = In_Place_Expenses_DF.at[' Total', 'Total'] / units





In_Place_Expenses_DF

,Per Unit,Total
Repairs & Maintenance,400,60000
Turnover,150,22500
Marketing,150,22500
General & Administrative,250,37500
Payroll,600,90000
Management Fees,380.34,57050.85
Utilities,300,45000
Real Estate Taxes,2666.67,400000
Property Insurance,200,30000
Capital Reserves,200,30000


# **MODEL**